<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 6.0  Live Stream (Optional)
### USB WEBCAM CONNECTED TO JETSON NANO REQUIRED

In the examples presented so far, the input stream has been a file, played as a stream.  In this notebook, you'll use a live stream via a webcam. Attach a USB webcam to your Jetson Nano using an available USB port.  

<img src="images/06_example.png" alt="webcam input">

6.1 **[Build a Pipeline with Webcam Input](#06_overview)**<br>
&nbsp; &nbsp; &nbsp; 6.1.1 [Practice Application `deepstream-test1-webcam_in`](#06_base)<br>
&nbsp; &nbsp; &nbsp; 6.1.2 [Exercise: Build and Run the Base Application](#06_ex_base)<br>
6.2 **[Change the Network to YOLO](#06_yolo)**<br>
&nbsp; &nbsp; &nbsp; 6.2.1 [Exercise: Run YOLO with a Webcam](#06_ex_yolo)<br>

<a name='06_overview'></a>
# 6.1 Build a Pipeline with Webcam Input
The `deepstream-test1-webcam_in` application is a modification of the `deepstream-test1-rstp_out` application you explored in the first notebook. Compare the pipeline elements defined in the C code in [filesrc deepstream_test_app.c](../deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-rtsp_out/deepstream_test1_app.c) with the elements in [the webcam deepstream_test1_app.c](../deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in/deepstream_test1_app.c):

**filesrc version**
```c
...

  GstElement *pipeline = NULL, *source = NULL, *h264parser = NULL,
      *decoder = NULL, *streammux = NULL, *sink = NULL, *pgie = NULL, *nvvidconv = NULL,
      *nvosd = NULL, *encoder = NULL, *rtppay = NULL, *transform = NULL, *cap_filter = NULL;

...
   /* Source element for reading from the file */
  source = gst_element_factory_make ("filesrc", "file-source");

  /* Since the data format in the input file is elementary h264 stream,
   * we need a h264parser */
  h264parser = gst_element_factory_make ("h264parse", "h264-parser");

  /* Use nvdec_h264 for hardware accelerated decode on GPU */
  decoder = gst_element_factory_make ("nvv4l2decoder", "nvv4l2-decoder");

  /* Create nvstreammux instance to form batches from one or more sources. */
  streammux = gst_element_factory_make ("nvstreammux", "stream-muxer");

...
```
**webcam version**
```c
...
    
  GstElement *pipeline = NULL, *source = NULL, *nvvidconv_src = NULL, *vidconv_src=NULL, 
      *filter_src=NULL, *streammux = NULL, *sink = NULL, *pgie = NULL, *nvvidconv = NULL,
      *nvosd = NULL, *encoder = NULL, *rtppay = NULL, *transform = NULL, *cap_filter = NULL;


...

  source = gst_element_factory_make ("v4l2src", "camera-source");
  g_object_set (G_OBJECT (source), "device", "/dev/video0", NULL);
  vidconv_src = gst_element_factory_make ("videoconvert", "vidconv_src");
  nvvidconv_src = gst_element_factory_make ("nvvideoconvert", "nvvidconv_src");
  filter_src = gst_element_factory_make ("capsfilter", "filter_src");
  g_object_set (G_OBJECT (nvvidconv_src), "nvbuf-memory-type", 0, NULL);
  caps_filter_src =
        gst_caps_from_string ("video/x-raw(memory:NVMM), format=NV12, width=1280, height=720, framerate=30/1");
  g_object_set (G_OBJECT (filter_src), "caps", caps_filter_src, NULL);
  gst_caps_unref (caps_filter_src);

  /* Create nvstreammux instance to form batches from one or more sources. */
  streammux = gst_element_factory_make ("nvstreammux", "stream-muxer");
```    

The remaining elements of the pipeline are the same.

In summary, the pipeline for this app consists of the following plugins (ordered):

- `GstV4l2Src` - can be used to capture video from v4l2 devices, like webcams and tv cards
- `GstVideoConvert` - Convert video frames between a great variety of video formats
- `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `GstH264Parse` - parses the incoming H264 stream
- `Gst-nvv4l2decoder` - hardware accelerated decoder; decodes video streams using NVDEC
- `Gst-nvstreammux` - batch video streams before sending for AI inference
- `Gst-nvinfer` - runs inference using TensorRT
- `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
- `Gst-nvdsosd` - draw bounding boxes, text and region of interest (ROI) polygons
- `Gst-nvvideoconvert` - performs video color format conversion (RGBA to I420)
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `Gst-nvv4l2h264enc` - encodes RAW data in I420 format to H264
- `GstRtpH264Pay` - converts H264 encoded Payload to RTP packets (RFC 3984)
- `GstUDPSink` - sends UDP packets to the network. When paired with RTP payloader (`Gst-rtph264pay`) it can implement RTP streaming

<a name='06_base'></a>
# 6.1.1 Practice Application `deepstream-test1-webcam_in`
This app uses the Resnet10 primary detector provided with the DeepStream SDK.  You'll need to move the webcam over objects that the network can detect, i.e. Vehicle, Person, Bicycle, or Roadsign.  One way to accomplish this is to use a computer screen with these objects visible.

<a name='06_ex_base'></a>
# 6.1.2 Exercise: Build and Run the Base Application
Plug in your webcam and execute the following cells. 

In [2]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in/
!make clean
!make

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in
rm -rf deepstream_test1_app.o deepstream-test1-app
cc -c -o deepstream_test1_app.o -DPLATFORM_TEGRA -I../../../includes `pkg-config --cflags gstreamer-1.0` deepstream_test1_app.c
cc -o deepstream-test1-app deepstream_test1_app.o `pkg-config --libs gstreamer-1.0` -L/opt/nvidia/deepstream/deepstream-4.0/lib/ -lnvdsgst_meta -lnvds_meta -lgstrtspserver-1.0 -Wl,-rpath,/opt/nvidia/deepstream/deepstream-4.0/lib/


#### Run the DeepStream app
Open the VLC media player on your laptop:
- Click "Media" and open the  "Open Network Stream" dialog.
- Set the URL to `rtsp://192.168.55.1:8554/ds-test`.
- Start execution of the cell below.
- Click "Play" on your VLC media player right after you start executing the cell.  

The stream will start shortly from the Jetson Nano and display in the media player.  You may experience some lag, or may need to press the play button again on the player.  

To stop the stream, click `Kernel -> Interrupt Kernel` from the JupyterLab menu.

In [3]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in/
!./deepstream-test1-app

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in

 *** DeepStream: Launched RTSP Streaming at rtsp://localhost:8554/ds-test ***

Now playing: (null)
Opening in BLOCKING MODE 
Creating LL OSD context new
Running...
Creating LL OSD context new
NvMMLiteOpen : Block : BlockType = 4 
===== NVMEDIA: NVENC =====
NvMMLiteBlockCreate : Block : BlockType = 4 
H264: Profile = 66, Level = 0 

(deepstream-test1-app:18564): GLib-GObject-WARNING **: 04:40:28.346: g_object_get_is_valid_property: object class 'GstUDPSrc' has no property named 'pt'
^C


<a name='06_yolo'></a>
# 6.2 Change the Network to YOLO
If you set up the YOLO network in the optional [Using Different Networks]() notebook, you will be able to identify many more objects with your webcam (up to 80!).  The `deepstream-test1-webcam_in-yolo` is very similar to the `deepstream-test1-webcam_in` app, but configured for YOLO.  To see the actual difference in lines, execute the following `diff` command.

In [2]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/
!diff deepstream-test1-webcam_in/deepstream_test1_app.c deepstream-test1-webcam_in-yolo/deepstream_test1_app.c

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps
286c286
<       "config-file-path", "dstest1_pgie_config.txt", NULL);
---
>       "config-file-path", "dstest1_pgie_config_yolov3_tiny.txt", NULL);
352,353c352,353
<     gst_pad_add_probe (osd_sink_pad, GST_PAD_PROBE_TYPE_BUFFER,
<         osd_sink_pad_buffer_probe, NULL, NULL);
---
>     //gst_pad_add_probe (osd_sink_pad, GST_PAD_PROBE_TYPE_BUFFER,
>     //    osd_sink_pad_buffer_probe, NULL, NULL);


<a name='06_ex_yolo'></a>
# 6.2.1 Run YOLO with a Webcam
There will be a delay while the `.engine` file is built.

In [1]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in-yolo/
!make clean
!make

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in-yolo
rm -rf deepstream_test1_app.o deepstream-test1-app
cc -c -o deepstream_test1_app.o -DPLATFORM_TEGRA -I../../../includes `pkg-config --cflags gstreamer-1.0` deepstream_test1_app.c
cc -o deepstream-test1-app deepstream_test1_app.o `pkg-config --libs gstreamer-1.0` -L/opt/nvidia/deepstream/deepstream-4.0/lib/ -lnvdsgst_meta -lnvds_meta -lgstrtspserver-1.0 -Wl,-rpath,/opt/nvidia/deepstream/deepstream-4.0/lib/


In [3]:
%cd /home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in-yolo/
!./deepstream-test1-app

/home/dlinano/deepstream_sdk_v4.0.2_jetson/sources/apps/dli_apps/deepstream-test1-webcam_in-yolo

 *** DeepStream: Launched RTSP Streaming at rtsp://localhost:8554/ds-test ***

Now playing: (null)
Opening in BLOCKING MODE 
Creating LL OSD context new
0:00:04.325434062 18945   0x55a7bef860 WARN                 nvinfer gstnvinfer.cpp:515:gst_nvinfer_logger:<primary-nvinference-engine> NvDsInferContext[UID 1]:useEngineFile(): Failed to read from model engine file
0:00:04.325497188 18945   0x55a7bef860 INFO                 nvinfer gstnvinfer.cpp:519:gst_nvinfer_logger:<primary-nvinference-engine> NvDsInferContext[UID 1]:initialize(): Trying to create engine from model files
Loading pre-trained weights...
Loading complete!
Total Number of weights read : 62001757
      layer               inp_size            out_size       weightPtr
(1)   conv-bn-leaky     3 x 608 x 608      32 x 608 x 608    992   
(2)   conv-bn-leaky    32 x 608 x 608      64 x 304 x 304    19680 
(3)   conv-bn-leaky    64

<h2 style="color:green;">Congratulations!</h2>

You've completed all the notebooks.  Be sure to work through the assessment questions in the online portion of the DLI course to get your certificate!

<center><img src="images/DLI Header.png" alt="Header" width="400"></center>